In [ ]:
from pathlib import Path
import csv
import datetime
import bisect
import time

import requests
from bs4 import BeautifulSoup

from stock.util import convert_to_number
import stock

In [ ]:
def get_stock_data(code: str, base_url: str = "https://kabutan.jp/stock/kabuka?code={}&ashi=day"):
    res = requests.get(base_url.format(code))
    soup = BeautifulSoup(res.text)

    daily_data = []
    stock_tables = soup.find("div", {"id": "stock_kabuka_table"})
    if stock_tables is None:
        return daily_data
    for table in stock_tables.find_all("table"):
        tbody = table.find("tbody")
        if tbody is None:
            continue
        for table_row in tbody.find_all("tr"):
            thead = table_row.find("th")
            tdata = table_row.find_all("td")
            if thead is None or len(tdata) < 7:
                break
            date = datetime.datetime.strptime(thead.text, "%y/%m/%d")
            start = convert_to_number(tdata[0].text)
            high = convert_to_number(tdata[1].text)
            low = convert_to_number(tdata[2].text)
            end = convert_to_number(tdata[3].text)
            volume = convert_to_number(tdata[6].text)
            daily_data.append([date, start, high, low, end, volume])
    return daily_data

In [ ]:
def update_csv(code: str = "1301"):
    csv_path = Path("../data/daily/{}.csv".format(code))
    data = []
    if csv_path.exists():
        with open(csv_path, "r") as f:
            csv_reader = csv.reader(f)
            next(csv_reader)
            data += [[row[0]] + [convert_to_number(col) for col in row[1:]] for row in csv_reader] 
    data = sorted(data, key=lambda x : x[0])
    num_data = len(data)

    new_data = get_stock_data(code)
    for d in new_data:
        if bisect.bisect_left(data, d[0].strftime("%Y/%m/%d"), key=lambda x: x[0]) == len(data):
            data.append([d[0].strftime("%Y/%m/%d")] + d[1:])

    if num_data < len(data):
        header = ["date", "open", "high", "low", "close", "volume"]
        with open(csv_path, "w", encoding="utf-8") as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(header)
            csv_writer.writerows(data)
        stock.logger.info(f"Update csv : {csv_path}")